In [750]:
# import the pre_defined files
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [751]:
# Reading the dataset
df=pd.read_excel('/content/Sample_Longlist_Data.xlsx')

In [752]:
# I want to see the head of the dataset
df.head()

,Date Added,category,sellerlink,sellerlink-url,sellerstorefront-url,sellerproductcount,sellerratings,sellerdetails,seller business name,businessaddress,Count of seller brands,Max % of negative seller ratings - last 30 days,Max % of negative seller ratings - last 90 days,Max % of negative seller ratings - last 12 months,Hero Product 1 #ratings,Hero Product 2 #ratings,Sample brand name,Sample Brand URL
0,2020-11-15,Garden,Seller 1,Seller 1-link,Seller 1-storefrontlink,"1-16 of over 100,000 results",88% positive in the last 12 months (118 ratings),Lohas Living Inc James Mazzello US 845 3RD Ave...,Business Name:Lohas Living Inc,411 THEODORE FREMD AVESTE 206 SOUTHRyeNY10580-...,16,17,11,12,5873,4957,Visit the YAMAZAKI Store,https://www.amazon.de/-/en/stores/YAMAZAKI/pag...
1,2020-11-15,Garden,Seller 2,Seller 2-link,Seller 2-storefrontlink,NaN,90% positive in the last 12 months (566 ratings),Herzlich Willkommen im Amazon Shop von 1a-Hand...,Business Name:1a-Handelsagentur,1a HandelsagenturLindenallee 2MalchowMecklenbu...,0,2,2,9,0,0,Visit the tesa Store,https://www.amazon.de/-/en/stores/tesa/page/12...
2,2020-11-15,Garden,Seller 3,Seller 3-link,Seller 3-storefrontlink,"1-16 of over 2,000 results",85% positive in the last 12 months (75 ratings),67878u6i7 is committed to providing each custo...,Business Name:nantongaohuanfangzhipinyouxiango...,zhongnanshijicheng27zhuang1410haochongchuanqun...,15,0,25,12,164,162,TXYFYP,https://www.amazon.de/-/en/s/ref=bl_dp_s_web_0...
3,2020-11-15,Garden,Seller 4,Seller 4-link,Seller 4-storefrontlink,1-16 of 123 results,NaN,7 days home life supermarket is committed to p...,Business Name:Zhengzhou Yaoqiong Trading Company,河南省中兴南路福禄街自贸试验区郑州片区（郑东）河南商会大厦1901室450000CN,1,0,0,0,0,0,SHIFENG,https://www.amazon.de/-/en/s/ref=bl_dp_s_web_0...
4,2020-11-15,Garden,Seller 5,Seller 5-link,Seller 5-storefrontlink,"1-16 of over 1,000 results",81% positive in the last 12 months (52 ratings),78 68yui678 is committed to providing each cus...,Business Name:shenzhenfeichangshangmeimaoyiyou...,minzhidadaoshaxialaocun94dongA901longhuaxinqus...,16,0,31,15,224,113,YUOKI99,https://www.amazon.de/-/en/s/ref=bl_dp_s_web_0...


In [753]:
#I want to see the tail of the dataset
df.tail()

,Date Added,category,sellerlink,sellerlink-url,sellerstorefront-url,sellerproductcount,sellerratings,sellerdetails,seller business name,businessaddress,Count of seller brands,Max % of negative seller ratings - last 30 days,Max % of negative seller ratings - last 90 days,Max % of negative seller ratings - last 12 months,Hero Product 1 #ratings,Hero Product 2 #ratings,Sample brand name,Sample Brand URL
1834,2020-11-15,Garden,Seller 1835,Seller 1835-link,Seller 1835-storefrontlink,"1-16 of over 2,000 results",83% positive in the last 12 months (46 ratings),ZZM Store is committed to providing each custo...,"Business Name:Shenzhen Xinyile Trading Co., Ltd",Longhuaxinqu Longhuajiedao YousongDiergongyequ...,5,0,9,15,95,76,ZZMUK,https://www.amazon.de/-/en/s/ref=bl_dp_s_web_0...
1835,2020-11-15,Garden,Seller 1836,Seller 1836-link,Seller 1836-storefrontlink,"1-16 of over 5,000 results",NaN,ZZMIARY is committed to providing each custome...,Business Name:wujixianzhanpingzongheshangdian,hebeishengshijiazhuangshiwujixiangaotouxiangxi...,0,0,0,0,1,0,Not Applicable,https://www.amazon.de/-/en/s/ref=bl_dp_s_web_0...
1836,2020-11-15,Garden,Seller 1837,Seller 1837-link,Seller 1837-storefrontlink,"1-16 of over 10,000 results",0% positive lifetime (1 total ratings),zzshd75 is committed to providing each custome...,Business Name:henanliyanshangmaoyouxiangongsi,lankaoxianzhangjunmuzhenzhengzhuangcunerzukaif...,10,0,0,0,1,1,zwbaobei,https://www.amazon.de/-/en/s/ref=bl_dp_s_web_0...
1837,2020-11-15,Garden,Seller 1838,Seller 1838-link,Seller 1838-storefrontlink,1-16 of 473 results,67% positive lifetime (3 total ratings),ZZY xiaodian is committed to providing each cu...,Business Name:shanximinsanyizhuangshijiancaiyo...,bei san huan da ming gong jian cai shi changba...,4,0,0,33,9,1,ZZYXiao,https://www.amazon.de/-/en/s/ref=bl_dp_s_web_0...
1838,2020-11-15,Garden,Seller 1839,Seller 1839-link,Seller 1839-storefrontlink,"1-16 of over 20,000 results",100% positive lifetime (2 total ratings),zzzswbl is committed to providing each custome...,Business Name:nanchangshiyunjianshejizhuangshi...,honggutanxinquhuanandongerlu99haolongxiangyuan...,1,0,0,0,1,1,zzzswbl,https://www.amazon.de/-/en/s/ref=bl_dp_s_web_0...


In [754]:
# I want to see the shape of the dataset
df.shape

(1839, 18)

# After careful observation i found these things I must do

In [755]:
# Preprocessing Steps
# Step 1 : To extract the final value in product count column
# Step2 : Extract the positive % value calculate on the given value
# Step 3 : Extract the seller E-mail ID's and Phone numbers
# Step 4: Extract the country and exclude the china sellers
# Step 5 : Finding the best selling product based on hero product rating

# STEP-1: To extract the final value in product count column

In [756]:
df['sellerproductcount']=df['sellerproductcount'].str.replace(',','')
df['sellerproductcount'] = df.sellerproductcount.str.extract('(\d+) results')

In [757]:
# sanity check the column
df['sellerproductcount']

0       100000
1          NaN
2         2000
3          123
4         1000
         ...  
1834      2000
1835      5000
1836     10000
1837       473
1838     20000
Name: sellerproductcount, Length: 1839, dtype: object

# STEP-2: Extract the positive % value calculate on the given value

In [758]:
df['sellerratings'] = df['sellerratings'].str.replace("%","")
# removing the % symbol in sellerratings column


In [759]:
df['sellerratings1']=df['sellerratings'].str.extract('(\d+)')
# after removing % then we are extracting the value(like 88 value is extracted insted of 88%)


In [760]:
# Now i want to drop null values and change the datatype
df['sellerratings1']=df['sellerratings1'].dropna(how='any').astype(int)

# Now I  want to extract the ratings 

In [761]:
# Now I am extract the rating values so I want to remove the '("   and putting # symbol for instead of "(" 
df['sellerratings12'] = df.sellerratings.str.replace('(','#')

In [762]:
# here I am extracting the value after # symbol
df['sellerratings121'] = df['sellerratings12'].str.extract(r'(?<=#)(\d+)')

In [763]:
# converting the data type
df['Total_ratings']=df['sellerratings121'].dropna(how='any').astype(int)

In [764]:
# Now I am calculating the positive percentage
# 88% means how much the value
df['Perc_Good'] = (df['sellerratings1'] /100) * df['Total_ratings']

In [765]:
# apply the floor value to Positive percentage
df['Number_of_Positive_Sellerratings']=df['Perc_Good'].apply(np.floor)

In [766]:
df['negative'] = df['Total_ratings']- df['Perc_Good']
# calculate negative rating also
df['Number_of_negative_Sellerratings']=df['negative'].apply(np.floor)
# applying the ceil value

# STEP-3: Extract the country and exclude the china sellers

In [767]:
# Extract the sellers register in which country and exclude the china sellers
k=[]
for  i in df['businessaddress']:
  i= str(i)
  k.append(i[-2:])
df['businessaddress'] =k
df = df[df.businessaddress != 'CN']

# STEP-4: Finding the best selling product based on hero product rating

In [768]:
# Here I am finding the Which product is having best ratings 
df['Which_product_having_best_rating'] = np.where(df['Hero Product 1 #ratings'] > df['Hero Product 2 #ratings'],'Hero Product 1','Hero Product 2')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [769]:
# Here I want to find the their respected ratings for each product
df['best_ratings_for_each_product'] = np.where(df['Hero Product 1 #ratings'] > df['Hero Product 2 #ratings'],df['Hero Product 1 #ratings'] , df['Hero Product 2 #ratings'])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


# STEP-5: Extract the seller E-mail ID's and Phone numbers

In [770]:
# finding the emails and adding to the dataframe
import re
def find_email(sellerdetails):
    email = re.findall(r'[\w\.-]+@[\w\.-]+',str(sellerdetails))
    return ",".join(email)
df['email']=df['sellerdetails'].apply(lambda x: find_email(x))
print("\Extracting email from dataframe columns:")
print(df['email'])

\Extracting email from dataframe columns:
0                                   jadgemaello@gmail.com
1       info@1a-handelsagentur.de,info@1a-handelsagent...
5                                                        
14                                 webmaster@9pm-store.de
19                                       abc-markt@web.de
                              ...                        
1808                                                     
1813                info@zurrosakuh.de,info@zurrosakuh.de
1814    r.joormann@googlemail.com,r.joormann@googlemai...
1815                                                     
1828                                                     
Name: email, Length: 602, dtype: object


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [771]:
# telefone having different names so i can change all names and give the single name
df['sellerdetails']=df.sellerdetails.str.replace('Telefonnummer:','Telefone')
df['sellerdetails']=df.sellerdetails.str.replace('telefonnummer:','Telefone')
df['sellerdetails']=df.sellerdetails.str.replace('telfone:','Telefone')
df['sellerdetails']=df.sellerdetails.str.replace('Tel.:','Telefone')
df['sellerdetails']=df.sellerdetails.str.replace('Tel:','Telefone')
df['sellerdetails']=df.sellerdetails.str.replace('tel:','Telefone')


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

In [772]:
# finding the Mobile numbers of the sellers
def find_telefone(sellerdetails):
    Telefone = re.findall(r'(?<=Telefone )(\d+)',str(sellerdetails))
    return ",".join(Telefone)
df['Telefone']=df['sellerdetails'].apply(lambda x:find_telefone(x))
print("\Extracting email from dataframe columns:")
print(df['Telefone'])

\Extracting email from dataframe columns:
0                 
1           039932
5                 
14                
19      0570591155
           ...    
1808              
1813              
1814              
1815              
1828              
Name: Telefone, Length: 602, dtype: object


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


# Completed the all steps in the assignmet and 

In [773]:
# Now removing the Unnecessary columns in the dataset
df.head()

,Date Added,category,sellerlink,sellerlink-url,sellerstorefront-url,sellerproductcount,sellerratings,sellerdetails,seller business name,businessaddress,Count of seller brands,Max % of negative seller ratings - last 30 days,Max % of negative seller ratings - last 90 days,Max % of negative seller ratings - last 12 months,Hero Product 1 #ratings,Hero Product 2 #ratings,Sample brand name,Sample Brand URL,sellerratings1,sellerratings12,sellerratings121,Total_ratings,Perc_Good,Number_of_Positive_Sellerratings,negative,Number_of_negative_Sellerratings,Which_product_having_best_rating,best_ratings_for_each_product,email,Telefone
0,2020-11-15,Garden,Seller 1,Seller 1-link,Seller 1-storefrontlink,100000,88 positive in the last 12 months (118 ratings),Lohas Living Inc James Mazzello US 845 3RD Ave...,Business Name:Lohas Living Inc,US,16,17,11,12,5873,4957,Visit the YAMAZAKI Store,https://www.amazon.de/-/en/stores/YAMAZAKI/pag...,88.0,88 positive in the last 12 months #118 ratings),118,118.0,103.84,103.0,14.16,14.0,Hero Product 1,5873,jadgemaello@gmail.com,
1,2020-11-15,Garden,Seller 2,Seller 2-link,Seller 2-storefrontlink,NaN,90 positive in the last 12 months (566 ratings),Herzlich Willkommen im Amazon Shop von 1a-Hand...,Business Name:1a-Handelsagentur,DE,0,2,2,9,0,0,Visit the tesa Store,https://www.amazon.de/-/en/stores/tesa/page/12...,90.0,90 positive in the last 12 months #566 ratings),566,566.0,509.40,509.0,56.60,56.0,Hero Product 2,0,"info@1a-handelsagentur.de,info@1a-handelsagent...",039932
5,2020-11-15,Garden,Seller 6,Seller 6-link,Seller 6-storefrontlink,28,NaN,7830jeff is committed to providing each custom...,NaN,an,13,0,0,0,3284,2944,Strata,https://www.amazon.de/-/en/s/ref=bl_dp_s_web_0...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Hero Product 1,3284,,
14,2020-11-15,Garden,Seller 15,Seller 15-link,Seller 15-storefrontlink,919,97 positive in the last 12 months (116 ratings),Impressum\n \n Gesetzliche Anbieterkennung:\n ...,Business Name:Oliver Mills,DE,15,0,0,3,1446,921,Unbekannt,https://www.amazon.de/-/en/Unbekannt/b/ref=bl_...,97.0,97 positive in the last 12 months #116 ratings),116,116.0,112.52,112.0,3.48,3.0,Hero Product 1,1446,webmaster@9pm-store.de,
19,2020-11-15,Garden,Seller 20,Seller 20-link,Seller 20-storefrontlink,NaN,76 positive in the last 12 months (3721 ratings),Impressum: ABC-Schnäppchenmarkt GmbH \n Jösser...,Business Name:ABC-Schnäppchenmarkt GmbH,DE,0,18,20,20,0,0,Visit the TFA Dostmann Store,https://www.amazon.de/-/en/stores/TFA+Dostmann...,76.0,76 positive in the last 12 months #3721 ratings),3721,3721.0,2827.96,2827.0,893.04,893.0,Hero Product 2,0,abc-markt@web.de,0570591155


In [774]:
de = ['sellerlink','sellerlink-url','sellerstorefront-url','sellerratings','sellerratings121','sellerdetails','seller business name','Sample brand name','Sample Brand URL','sellerratings12','Perc_Good','negative',]

# Here I am dropping the columns Based on assignment only

In [775]:
# droping the columns
df.drop(de,axis=1 , inplace=True)

In [776]:
df=df.rename(columns={'Date Added': 'Date_added','category':'Category','Count of seller brands':'Count_of_seller_brands','Max % of negative seller ratings - last 30 days':'Max_%_of_negative_seller_ratings_last_30_days','Max % of negative seller ratings - last 90 days':'Negative%_of _last_90_days','Max % of negative seller ratings - last 12 months' :'Last_12_months_negative%'})

In [777]:
df.head()

,Date_added,Category,sellerproductcount,businessaddress,Count_of_seller_brands,Max_%_of_negative_seller_ratings_last_30_days,Negative%_of _last_90_days,Last_12_months_negative%,Hero Product 1 #ratings,Hero Product 2 #ratings,sellerratings1,Total_ratings,Number_of_Positive_Sellerratings,Number_of_negative_Sellerratings,Which_product_having_best_rating,best_ratings_for_each_product,email,Telefone
0,2020-11-15,Garden,100000,US,16,17,11,12,5873,4957,88.0,118.0,103.0,14.0,Hero Product 1,5873,jadgemaello@gmail.com,
1,2020-11-15,Garden,NaN,DE,0,2,2,9,0,0,90.0,566.0,509.0,56.0,Hero Product 2,0,"info@1a-handelsagentur.de,info@1a-handelsagent...",039932
5,2020-11-15,Garden,28,an,13,0,0,0,3284,2944,NaN,NaN,NaN,NaN,Hero Product 1,3284,,
14,2020-11-15,Garden,919,DE,15,0,0,3,1446,921,97.0,116.0,112.0,3.0,Hero Product 1,1446,webmaster@9pm-store.de,
19,2020-11-15,Garden,NaN,DE,0,18,20,20,0,0,76.0,3721.0,2827.0,893.0,Hero Product 2,0,abc-markt@web.de,0570591155


In [778]:
# I want to find the null values in the dataset
df.isnull().sum()

Date_added                                         0
Category                                           0
sellerproductcount                               121
businessaddress                                    0
Count_of_seller_brands                             0
Max_%_of_negative_seller_ratings_last_30_days      0
Negative%_of _last_90_days                         0
Last_12_months_negative%                           0
Hero Product 1 #ratings                            0
Hero Product 2 #ratings                            0
sellerratings1                                    43
Total_ratings                                     43
Number_of_Positive_Sellerratings                  43
Number_of_negative_Sellerratings                  43
Which_product_having_best_rating                   0
best_ratings_for_each_product                      0
email                                              0
Telefone                                           0
dtype: int64

In [779]:
# Now I am converting the data type of sellerproductcount column
df['sellerproductcount']=round(df.sellerproductcount.astype(float),0)

# Treatment With the missing values

In [780]:
# filling the missing values
miss_values= ['sellerproductcount','sellerratings1','Total_ratings','Number_of_Positive_Sellerratings','Number_of_negative_Sellerratings']

In [781]:
# filling all columns all missing values
for i in miss_values:
  df[i]=df[i].fillna(df[i].mean())

In [782]:
#Know checking 
df.isnull().sum()

Date_added                                       0
Category                                         0
sellerproductcount                               0
businessaddress                                  0
Count_of_seller_brands                           0
Max_%_of_negative_seller_ratings_last_30_days    0
Negative%_of _last_90_days                       0
Last_12_months_negative%                         0
Hero Product 1 #ratings                          0
Hero Product 2 #ratings                          0
sellerratings1                                   0
Total_ratings                                    0
Number_of_Positive_Sellerratings                 0
Number_of_negative_Sellerratings                 0
Which_product_having_best_rating                 0
best_ratings_for_each_product                    0
email                                            0
Telefone                                         0
dtype: int64

In [783]:
df.head()

,Date_added,Category,sellerproductcount,businessaddress,Count_of_seller_brands,Max_%_of_negative_seller_ratings_last_30_days,Negative%_of _last_90_days,Last_12_months_negative%,Hero Product 1 #ratings,Hero Product 2 #ratings,sellerratings1,Total_ratings,Number_of_Positive_Sellerratings,Number_of_negative_Sellerratings,Which_product_having_best_rating,best_ratings_for_each_product,email,Telefone
0,2020-11-15,Garden,100000.000000,US,16,17,11,12,5873,4957,88.000000,118.000000,103.000000,14.000000,Hero Product 1,5873,jadgemaello@gmail.com,
1,2020-11-15,Garden,6538.110187,DE,0,2,2,9,0,0,90.000000,566.000000,509.000000,56.000000,Hero Product 2,0,"info@1a-handelsagentur.de,info@1a-handelsagent...",039932
5,2020-11-15,Garden,28.000000,an,13,0,0,0,3284,2944,90.266547,462.515206,431.468694,30.266547,Hero Product 1,3284,,
14,2020-11-15,Garden,919.000000,DE,15,0,0,3,1446,921,97.000000,116.000000,112.000000,3.000000,Hero Product 1,1446,webmaster@9pm-store.de,
19,2020-11-15,Garden,6538.110187,DE,0,18,20,20,0,0,76.000000,3721.000000,2827.000000,893.000000,Hero Product 2,0,abc-markt@web.de,0570591155


# Now I am finding the best sellers based on Number of products selling and Positive rating

In [784]:
# finding the top 5 best sellers
df['top']= round((df['sellerproductcount'] * df['Number_of_negative_Sellerratings'])/2,2)

In [785]:
# sorting the values
df.sort_values('top',ascending=False)

,Date_added,Category,sellerproductcount,businessaddress,Count_of_seller_brands,Max_%_of_negative_seller_ratings_last_30_days,Negative%_of _last_90_days,Last_12_months_negative%,Hero Product 1 #ratings,Hero Product 2 #ratings,sellerratings1,Total_ratings,Number_of_Positive_Sellerratings,Number_of_negative_Sellerratings,Which_product_having_best_rating,best_ratings_for_each_product,email,Telefone,top
712,2020-11-15,Garden,200000.0,ES,15,16,22,22,13490,11284,76.0,2251.0,1710.0,540.0,Hero Product 1,13490,,,54000000.0
23,2020-11-15,Garden,100000.0,DE,15,15,12,10,30139,11182,88.0,3134.0,2757.0,376.0,Hero Product 1,30139,bib.beckum@blumenbecker.com,,18800000.0
726,2020-11-15,Garden,40000.0,DE,16,8,9,9,40681,13311,89.0,7242.0,6445.0,796.0,Hero Product 1,40681,support@trendmile.com,,15920000.0
1007,2020-11-15,Garden,60000.0,HK,13,2,2,4,560,481,94.0,7465.0,7017.0,447.0,Hero Product 1,560,,,13410000.0
36,2020-11-15,Garden,50000.0,DE,15,7,8,8,40683,18910,91.0,5150.0,4686.0,463.0,Hero Product 1,40683,info@itenga.de,06825,11575000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
913,2020-11-15,Garden,226.0,DE,12,0,0,0,24,23,100.0,24.0,24.0,0.0,Hero Product 1,24,amazon@vonhermine.de,,0.0
157,2020-11-15,Garden,2000.0,DE,15,0,0,0,2036,1525,100.0,576.0,576.0,0.0,Hero Product 1,2036,kontakt@kochmesser-shop.com,0511,0.0
213,2020-11-15,Garden,84.0,PL,13,0,0,0,1538,581,100.0,2.0,2.0,0.0,Hero Product 1,1538,,,0.0
208,2020-11-15,Garden,836.0,DE,14,0,0,0,11917,5800,100.0,35.0,35.0,0.0,Hero Product 1,11917,buchfink.brigitte.fink@googlemail.comUmsatzste...,04443,0.0


In [786]:
# taking the top 5 best sellers only
df['top'].nlargest(n=5)

712     54000000.0
23      18800000.0
726     15920000.0
1007    13410000.0
36      11575000.0
Name: top, dtype: float64

# The above sellers are the best sellers in the given data

#                             Thank you